In [1]:
from functools import total_ordering, reduce
import csv
import re
import pandas as pd
import gensim
import math
import json
import pandas as pd

In [27]:
def vectorize(docID, vocab):
    vector = [0] * len(vocab)
    text = preprocess_doc(docs[docID])
    for t in tokenize(text):
        idx = vocab.index(t)
        vector[idx] = postings[t][docID] * vocab_idf[t]
    print(vector)
    
def CosineScore(q, docs, vocab_idf, postings):
    scores = [0 for d in docs]
    lenght = [len(d) for d in docs]
    for t in tokenize(q):
        if t in vocab_idf:
            w = vocab_idf[t]
            posting_list = postings[t]
            for doc, term_freq in posting_list.items():
                scores[doc] += w * term_freq
    scores = [scores[i] / lenght[i] for i in range(len(scores))]
    return scores        

In [14]:
df = pd.read_csv("wiki_movie.csv")
docs = df["Plot"]
docs = docs[:1000]

In [15]:
to_lower = lambda x: x.lower()

def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text

remove_extra_spaces = lambda x: re.sub(' +',' ',x)

def tokenize(text):
    return list(text.split())

def preprocess_doc(d):
    return remove_extra_spaces(clean_text(to_lower(d)))

def preprocess_query(q):
    return remove_extra_spaces(clean_text(to_lower(q)))

In [16]:
vocab = []
postings = {}

for i, d in enumerate(docs):
    if i % 100 == 0:
        print(f"processed {i} docs")
    processed_d = preprocess_doc(d)
    words = tokenize(processed_d)
    for word in words:
        if word not in vocab:
            vocab.append(word)
        wordId = vocab.index(word)
        wordId = word
        if wordId not in postings:
            postings[wordId] = {i:1}
        else:
            if i in postings[wordId]:
                postings[wordId][i] += 1
            else:
                postings[wordId][i] = 1

processed 0 docs
processed 100 docs
processed 200 docs
processed 300 docs
processed 400 docs
processed 500 docs
processed 600 docs
processed 700 docs
processed 800 docs
processed 900 docs


In [24]:
print(len(vocab))

16192


In [28]:
vocab_idf = {}
N = len(docs)

for v in vocab:
    vocab_idf[v] = math.log(N / len(postings[v]), 10)

In [29]:
myquery = "bartender with a beer"
q = preprocess_query(myquery)
print(q)

bartender with a beer


In [32]:
scores = CosineScore(q, docs, vocab_idf, postings)

1000
